In [1]:
import camelot
import os
import csv
import pdfplumber
import re

C:\Users\jaakd\PycharmProjects\dep1-24-25-g02\venv\Lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [2]:
file_path = "octa+-smart-variabel-2024-08.pdf"

In [3]:
csv_file = "octaplus_smartvariabel.csv"

In [4]:
contract_key = "OCTAPLUS_SMARTVARIABEL"

In [5]:
data = dict()

In [6]:
tables = camelot.read_pdf(file_path, pages="all", flavor="hybrid")

Overwriting cache for 0 419
Overwriting cache for 0 419
Overwriting cache for 0 419


In [7]:
for i, table in enumerate(tables):
    df = table.df
    if df.isin(["Vaste vergoeding (€/jaar)"]).any().any():
        fixed_fee_row = df.index[df.isin(["Vaste vergoeding (€/jaar)"]).any(axis=1)][0]
        number = re.findall(r'-?\d+,\d+', df.loc[fixed_fee_row][1])[0].replace(",", '.')
        data['AdministrativeCosts'] = float(number)

In [8]:
for i, table in enumerate(tables):
    df = table.df
    if df.isin(["Enkelvoudige meter"]).any().any():
        fixed_fee_row = df.index[df.isin(["Enkelvoudige meter"]).any(axis=1)][0]
        data["SingleMeterFixed"] = df.loc[fixed_fee_row][1]
    if df.isin(["Piekuren"]).any().any():
        fixed_fee_row = df.index[df.isin(["Piekuren"]).any(axis=1)][0]
        data["DualMeterDayFixed"] = df.loc[fixed_fee_row][1]
    if df.isin(["Daluren"]).any().any():
        fixed_fee_row = df.index[df.isin(["Daluren"]).any(axis=1)][0]
        data["DualMeterNightFixed"] = df.loc[fixed_fee_row][1]
    if df.isin(["Uitsluitend nachtmeter"]).any().any():
        fixed_fee_row = df.index[df.isin(["Uitsluitend nachtmeter"]).any(axis=1)][0]
        data["ExclusiveNightMeterFixed"] = df.loc[fixed_fee_row][1]

In [9]:
flatten = lambda *n: (e for a in n for e in (flatten(*a) if isinstance(a, (tuple, list)) else (a,)))

In [10]:
with pdfplumber.open(file_path) as pdf:
    tables = []
    for page in pdf.pages:
        tables.append(tables.append(page.extract_tables()))
    cells = list(flatten(tables))

In [11]:
with pdfplumber.open(file_path) as pdf:
    for page in pdf.pages:
        if "Kosten WKK" in page.extract_text():
            for line in page.extract_text().split("\n"):
                if "Kosten groene stroom " in line:
                    numbers = re.findall(r'-?\d+,\d+', line)
                    numbers = [float(num.replace(',', '.')) for num in numbers]
                    data['GreenElectricity']= numbers[-1]
                if "Kosten WKK" in line:
                    numbers = re.findall(r'-?\d+,\d+', line)
                    numbers = [float(num.replace(',', '.')) for num in numbers]
                    data['WKK'] = numbers[-1]

In [12]:
data

{'AdministrativeCosts': 65.0,
 'SingleMeterFixed': '7,78',
 'ExclusiveNightMeterFixed': '7,26',
 'GreenElectricity': 1.166,
 'WKK': 0.43}

In [13]:
year, month = re.search(r'(\d{4})-(\d{2})', file_path).groups()
date_key = f"{year}{month}01"  

In [14]:
with pdfplumber.open(file_path) as pdf:
    for page in pdf.pages:
        if "De elektriciteitsprijs voor OCTA+ Smart Variabel wordt maandelijks" in page.extract_text():
            text = page.extract_text().replace("\n", " ")
            # Afname
            data["SingleMeterVariableMeterFactor"], data["SingleMeterVariableBalancingCost"] = re.findall(
                r'\d+(?:,\d+)?', re.search(r'Enkelvoudige meter :\s*Belpex RLP \* [\d,]+ \+ [\d,]+', text).group(0))
            data["DualMeterDayVariableMeterFactor"], data["DualMeterDayVariableBalancingCost"] = re.findall(
                r'\d+(?:,\d+)?',
                re.search(r'Tweevoudige meter – Piek :\s*Belpex RLP \* [\d,]+ \+ [\d,]+', text).group(0))
            data["DualMeterNightVariableMeterFactor"], data["DualMeterNightVariableBalancingCost"] = re.findall(
                r'\d+(?:,\d+)?',
                re.search(r'Tweevoudige meter – Dal :\s*Belpex RLP \* [\d,]+ \+ [\d,]+', text).group(0))
            data["ExclusiveNightMeterVariableMeterFactor"], data[
                "ExclusiveNightMeterVariableBalancingCost"] = re.findall(r'\d+(?:,\d+)?', re.search(
                r'Exclusief Nachtmeter :\s*Belpex RLP \* [\d,]+ \+ [\d,]+', text).group(0))

            # Injectie
            try:
                data["SingleMeterInjectionMeterFactor"], data["SingleMeterInjectionBalancingCost"] = re.findall(
                        r'\d+(?:,\d+)?', re.search(r'enkelvoudig :\s*Belpex \* [\d,]+ \- [\d,]+', text).group(0))
                data["SingleMeterInjectionBalancingCost"] = -float(
                        data["SingleMeterInjectionBalancingCost"].replace(",", "."))

                data["DualMeterDayInjectionMeterFactor"], data["DualMeterDayInjectionBalancingCost"] = re.findall(
                        r'\d+(?:,\d+)?', re.search(r'tweevoudig/piekuren : Belpex \* [\d,]+ \- [\d,]+', text).group(0))
                data["DualMeterDayInjectionBalancingCost"] = -float(
                        data["DualMeterDayInjectionBalancingCost"].replace(",", "."))

                data["DualMeterNightInjectionMeterFactor"], data[
                        "DualMeterNightInjectionBalancingCost"] = re.findall(r'\d+(?:,\d+)?', re.search(
                        r'tweevoudig/daluren : Belpex \* [\d,]+ \- [\d,]+', text).group(0))
                data["DualMeterNightInjectionBalancingCost"] = -float(
                        data["DualMeterNightInjectionBalancingCost"].replace(",", "."))
            except:
                print(f"Datekey {date_key} failed, taking default values instead")
                data["SingleMeterInjectionMeterFactor"] = data["DualMeterDayInjectionMeterFactor"] = data["DualMeterNightInjectionMeterFactor"] = 0.915
                data["SingleMeterInjectionBalancingCost"] = data["DualMeterDayInjectionBalancingCost"] = data["DualMeterNightInjectionBalancingCost"] = -19.83

Datekey 20240801 failed, taking default values instead


In [15]:
file_exists = os.path.isfile(csv_file)

In [16]:
data = {key: value.replace(',', '.') if isinstance(value, str) else value for key, value in data.items()}

In [17]:
with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=';')
    
    # Write header only if file doesn't exist
    if not file_exists:
        header = ["DateKey", "ContractKey", "SingleMeterFixed", "DualMeterDayFixed", 
                  "DualMeterNightFixed", "ExclusiveNightMeterFixed", "SingleMeterVariableMeterFactor", 
                  "SingleMeterVariableBalancingCost", "DualMeterDayVariableMeterFactor", 
                  "DualMeterDayVariableBalancingCost", "DualMeterNightVariableMeterFactor", 
                  "DualMeterNightVariableBalancingCost", "ExclusiveNightMeterVariableMeterFactor", 
                  "ExclusiveNightMeterVariableBalancingCost", "DynamicMeterCost", "DynamicBalancingCost", 
                  "SingleMeterInjectionMeterFactor", "SingleMeterInjectionBalancingCost", 
                  "DualMeterDayInjectionMeterFactor", "DualMeterDayInjectionBalancingCost", 
                  "DualMeterNightInjectionMeterFactor", "DualMeterNightInjectionBalancingCost", 
                  "AdministrativeCosts", "GreenElectricity", "WKK"]
        writer.writerow(header)
    
    # Prepare data row in the specified order
    row_data = [
        date_key,
        contract_key,
        data.get('SingleMeterFixed', ''),
        data.get('DualMeterDayFixed', ''),
        data.get('DualMeterNightFixed', ''),
        data.get('ExclusiveNightMeterFixed', ''),
        data.get('SingleMeterVariableMeterFactor', ''),
        data.get('SingleMeterVariableBalancingCost', ''),
        data.get('DualMeterDayVariableMeterFactor', ''),
        data.get('DualMeterDayVariableBalancingCost', ''),
        data.get('DualMeterNightVariableMeterFactor', ''),
        data.get('DualMeterNightVariableBalancingCost', ''),
        data.get('ExclusiveNightMeterVariableMeterFactor', ''),
        data.get('ExclusiveNightMeterVariableBalancingCost', ''),
        data.get('DynamicMeterCost', ''),
        data.get('DynamicBalancingCost', ''),
        data.get('SingleMeterInjectionMeterFactor', ''),
        data.get('SingleMeterInjectionBalancingCost', ''),
        data.get('DualMeterDayInjectionMeterFactor', ''),
        data.get('DualMeterDayInjectionBalancingCost', ''),
        data.get('DualMeterNightInjectionMeterFactor', ''),
        data.get('DualMeterNightInjectionBalancingCost', ''),
        data.get('AdministrativeCosts', ''),
        data.get('GreenElectricity', ''),
        data.get('WKK', '')
    ]

    
    # Write data row
    writer.writerow(row_data)

print(f"Data for {date_key} successfully written to {csv_file}")

Data for 20240801 successfully written to octaplus_smartvariabel.csv
